In [27]:
import requests
import re
import numpy as np

In [28]:
urls = []
url_template = 'https://api.gokulnath.com/thirukkuralchapters/%d/thirukkurals'
paals = [[1,134]]
#paals = [[109,134]]
for each in paals:
	start = each[0]
	end = each[1]
	for x in range(start, end):
		urls.append(url_template%x)

In [29]:
def get_kurals(url):
	return [ x['Tamil'] for x in requests.get(url).json()['Data'] ]


In [30]:
def get_all_kurals()
    data = []
    for url in urls:
        res = get_kurals(url)
        _ = data.extend(res)
fname = '/Users/saravana/AI/datasets/tirukural_all.json'
f = open(fname, 'r')
data = simplejson.load(f)
f.close()

In [31]:
def cleanhtml(raw_html):
	cleanr = re.compile('<.*?>')
	cleantext = re.sub(cleanr, ' ', raw_html)
	return cleantext



In [32]:
data_1 = []
for each in data:
    data_1.append(cleanhtml(each))

In [33]:
data = data_1
import simplejson
f=open('/Users/saravana/AI/datasets/tirukural_all.json','w')
simplejson.dump(data,f)
f.close()

In [34]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam

In [35]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)

In [36]:
total_words = (len(tokenizer.word_index)) + 1

In [37]:
print(total_words)

6578


In [38]:
line=data[0]


In [39]:
tokenizer.texts_to_sequences([line])

[[1106, 1107, 1108, 1109, 1110, 1111, 43]]

In [40]:
token_list = tokenizer.texts_to_sequences([line])[0]

In [41]:
input_sequences = []
for line in data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_seq = token_list[:i+1]
        input_sequences.append(n_gram_seq)

In [42]:
len(input_sequences)

7983

In [43]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [44]:
print(len(ys[0]))


6578


In [45]:
print(tokenizer.word_index)

{'படும்': 1, 'தரும்': 2, 'கெடும்': 3, 'செயல்': 4, 'தலை': 5, 'கொளல்': 6, 'காமம்': 7, 'பெறின்': 8, 'பவர்': 9, 'தவர்': 10, 'விடல்': 11, 'யவர்': 12, 'விடும்': 13, 'இன்னா': 14, 'கண்': 15, 'எல்லாம்': 16, 'கடை': 17, 'இன்பம்': 18, 'தற்று': 19, 'செயின்': 20, 'கண்ணும்': 21, 'வேண்டும்': 22, 'நன்று': 23, 'நட்பு': 24, 'தில்': 25, 'வார்': 26, 'இல்': 27, 'தார்': 28, 'பெருமை': 29, 'வரும்': 30, 'வேந்தன்': 31, 'அரிது': 32, 'ஆக்கம்': 33, 'பிற': 34, 'துணை': 35, 'எஞ்ஞான்றும்': 36, 'ஆற்றின்': 37, 'இன்மை': 38, 'கேண்மை': 39, 'நெஞ்சு': 40, 'என்னும்': 41, 'மன்னவன்': 42, 'உலகு': 43, 'துடைத்து': 44, 'என்று': 45, 'வாழ்க்கை': 46, 'செய்யும்': 47, 'கில்லை': 48, 'என்ப': 49, 'இனிது': 50, 'சொல்': 51, 'கொல்லோ': 52, 'செல்வம்': 53, 'வினை': 54, 'பேதை': 55, 'நன்றி': 56, 'பேதைமை': 57, 'நோய்': 58, 'இடும்பை': 59, 'தில்லை': 60, 'கல்லா': 61, 'ஆன்ற': 62, 'செய்யாமை': 63, 'சான்றோர்': 64, 'நாடு': 65, 'படை': 66, 'தரண்': 67, 'காணின்': 68, 'காதலர்': 69, 'உயிர்க்கு': 70, 'கேடு': 71, 'வேண்டா': 72, 'தொடர்பு': 73, 'நாடி': 74, 'செயினும்': 75

In [46]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics = ['accuracy'] )


In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 100)            657800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300)               301200    
_________________________________________________________________
dense_1 (Dense)              (None, 6578)              1979978   
Total params: 2,938,978
Trainable params: 2,938,978
Non-trainable params: 0
_________________________________________________________________


In [48]:
history = model.fit(xs, ys, epochs=100, verbose=1)
model.save('/Users/saravana/AI/models/tirukural_with_3_paals.h5')

Train on 7983 samples
Epoch 1/100
7983/7983 [==============================] - 14s 2ms/sample - loss: 8.6756 - accuracy: 0.0026
Epoch 2/100
7983/7983 [==============================] - 11s 1ms/sample - loss: 7.9684 - accuracy: 0.0123
Epoch 3/100
7983/7983 [==============================] - 11s 1ms/sample - loss: 6.0085 - accuracy: 0.0722
Epoch 4/100
7983/7983 [==============================] - 11s 1ms/sample - loss: 2.9578 - accuracy: 0.3958
Epoch 5/100
7983/7983 [==============================] - 11s 1ms/sample - loss: 0.9567 - accuracy: 0.8169
Epoch 6/100
7983/7983 [==============================] - 12s 1ms/sample - loss: 0.3809 - accuracy: 0.9366
Epoch 7/100
7983/7983 [==============================] - 11s 1ms/sample - loss: 0.1787 - accuracy: 0.9709
Epoch 8/100
7983/7983 [==============================] - 12s 1ms/sample - loss: 0.1163 - accuracy: 0.9790
Epoch 9/100
7983/7983 [==============================] - 12s 1ms/sample - loss: 0.0861 - accuracy: 0.9822
Epoch 10/100
7983/7983 [

In [49]:
import tensorflow as tf

model = tf.keras.models.load_model('/Users/saravana/AI/models/tirukural_with_3_paals.h5')


In [86]:
def get_ai_kural(seed_text):
    next_words = 6
    for x in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ''
        for word,index in tokenizer.word_index.items():
            if index==predicted:
                output_word = word
                break
        seed_text = seed_text + ' ' + output_word
    return seed_text

In [87]:
get_ai_kural('உலகு')

'உலகு டுயிரிடை செய்யினும் நோநொந் தறனல்ல சால்பின் தறிவு'

In [88]:
get_ai_kural('இறைவன்')

'இறைவன் செறினுஞ்சீர் டொட்ட லரிது சொல்வன்மை தூதுரைப்பார்க் லரிது'

In [89]:
get_ai_kural('தீண்டா')

'தீண்டா காணப் படாஅதி குய்த்துரைக்க லான் முயக்கு கண்'

In [90]:
get_ai_kural('மக்கள்')

'மக்கள் அறிவுடைமை மாநிலத்து மன்னுயிர்க் கெல்லாம் நீக்கும் வென்றிடினும்'

In [91]:
get_ai_kural('கேளா')

'கேளா நல்ல லுடையுழி எஞ்ஞான்றும் காமம் நனவினான் பிற'

In [92]:
get_ai_kural('சான்றோன்')

'சான்றோன் ஆற்ற யாக்கை பொறுத்த நிலம் குறித்தது றுண்டு'

In [93]:
get_ai_kural('தாய்')

'தாய் டுயிரிடை மன்னவன் குடிகாத்தல் கற்றறிதல் ஆள்வினையோ டைந்துடன்'

In [94]:
get_ai_kural('மகன்')

'மகன் தந்தைக் காற்றும் உதவி இவன்தந்தை என்நோற்றான் கொல்லெனும்'

In [95]:
get_ai_kural('ஒழுக்கம்')

'ஒழுக்கம் விழுப்பந் தரலான் மன்னுயிர்க் காக்கம் இனநலம் எல்லாப்'

In [96]:
get_ai_kural('பார்ப்பான்')

'பார்ப்பான் செங்கோல் குடியோம்பல் கெடலும் நணித்து அரிது கீழ்'

In [97]:
get_ai_kural('அழுக்கா')

'அழுக்கா றெனவொரு பாவி ஒருவற்கு வயிற்றுக்கும் யானோ இன்றி'

In [98]:
get_ai_kural('மேன்மை')

'மேன்மை இழுக்கத்தின் தானே நன்றால் வற்றே அறிந்தியாக்க நட்பு'

In [99]:
get_ai_kural('தீய')

'தீய பயத்தலால் தீயவை தீயினும் அஞ்சப் படும் செருக்கு'

In [100]:
get_ai_kural('கல்லா')

'கல்லா ஒருவன் தகைமை தலைப்பெய்து சொல்லாடச் சோர்வு படும்'

In [101]:
get_ai_kural('ஒருவன்')

'ஒருவன் குடிமைக்கண் தங்கிற்றன் னென்னார்க் ஆன்ற துணையிலன் செயல்'

In [102]:
get_ai_kural('வையம்')

'வையம் எல்லாம் நகலானாம் நன்னயம் என்னும் செருக்கு ஒன்று'

In [103]:
get_ai_kural('அஞ்சாமை')

'அஞ்சாமை ஈகை அறிவூக்கம் இந்நான்கும் எஞ்சாமை வேந்தற் கியல்பு'

In [104]:
get_ai_kural('பெருமை')

'பெருமை சிறுமைதான் சுற்றமாச் சூழ்ந்து விடும் லதனை அறம்'

In [105]:
get_ai_kural('மாக்கள்')

'மாக்கள் தில்லார் தங்குற்றங் சொல்லா வாணிகம் பேணிப் பிறவும்'

In [106]:
get_ai_kural('குற்றம்')

'குற்றம் பெருகும் செய்யற்க தான்பிறி மாதர் முகத்து நிரப்பு'

In [107]:
get_ai_kural('யானை')

'யானை நன்றே தரினும் நடுவிகந்தாம் உகாஅமை கெழுதகைமை கேடு'

In [108]:
get_ai_kural('குற்றம்')

'குற்றம் பெருகும் செய்யற்க தான்பிறி மாதர் முகத்து நிரப்பு'

In [73]:
get_ai_kural('தீங்கு')

'தீங்கு முத்தம் தலைப்பிரியாச் சொல் செயினும் இகல்கருதி இன்னாசெய் துடைத்து யவை யாங்கு படும் வரும் அறநாணத் வினை யவர் படும் தரண் யவர் யவர் வணங்கிய யவர்'

In [109]:
get_ai_kural('போர்')

'போர் யாமத்தும் உள்ளுவேன் மன்ற படலொல்லா பேதைக்கென் கண்'

In [110]:
get_ai_kural('நட்பு')

'நட்பு டுயிரிடை கேளாத் தகையவே கேள்வியால் தோட்கப் தவர்'

In [111]:
get_ai_kural('தீங்கு')

'தீங்கு முத்தம் தலைப்பிரியாச் சொல் செயினும் இகல்கருதி இன்னாசெய்'

In [112]:
get_ai_kural('காலை')

'காலை அரும்பிப் இன்சொல் இருவினையும் நன்றி பயப்பதாந் தூது'

In [78]:
get_ai_kural('நன்றி')

'நன்றி மறப்பது நன்றன்று நன்றல்லது அன்றே மறப்பது நன்று கதவு செயினும் யவை கொளல் செயல் தரும் செயின் தரும் டெவன்கொல் தவர் யவர் அரிது பிற விடும்'

In [79]:
get_ai_kural('நோக்கும்')

'நோக்கும் பலர் தோன்றும் அசைவின்மை கைப்பொருள் போகூழால் தோன்று துணை தரும் படாஅர் சொல் தவர் யவை யவை யவர் பக்கு விடும் செயின் ஒண்பொருள் தவர் நட்பு'

In [80]:
get_ai_kural('என்ன')

'என்ன பேதைமை அஞ்சுவ தஞ்சல் அறிவார் றன்று ஆறு ஆறு கெடும் தரும் யவை ஆன்ற படும் நாடு யவர் யவர் யவர் தரும் அரிது தரும் யவை'

In [81]:
get_ai_kural('தலை')

'தலை டுயிரிடை செய்யினும் நோநொந் தறனல்ல செய்யாமை ஊன்றின் பவர் நோய் யவர் பவர்க்கு யவை பவர்க்கு யவை பவர்க்கு தரும் யவர் என்று படும் செயின் அரிது'

In [82]:
get_ai_kural('தோன்று')

'தோன்று எனப்படுவ ஆற்றின் நிறைந்த மன்னோ முயிர்க்கு பின் பழி யவை உடம்பிடும்பைக் நிரந்தினிது கண்இணை யவர் யவர் செயின் தலை யவை யவை சொல் நட்பு தரும்'

In [83]:
get_ai_kural('மகளிர்')

'மகளிர் கடிந்தொரார் செய்தார்க் கவைதாம் முடிந்தாலும் பீழை தரும் கதவு யவை யவர் தரும் யவை நீத்தாரு படும் யவை செயின் அரிது நட்பு தரும் பவர்க்கு விடும்'